# Laboratorio 4

1. Utilice el archivo generado en el laboratorio 3 con los documentos de la colección de CACM preprocesados.
1. Obtengan el vocabulario de la colección utilizando truncamiento. Impriman la longitud del vocabulario. Después de observar el vocabulario resultante, proponga mecanismos para reducirlo. Codifique los mecanismos propuestos. Escriba la longitud del vocabulario reducido. Las longitudes deberán escribirse a pantalla
1. Escriban los vocabularios finales a archivos vocabulario.txt y vocabulario_reducido.tx



## PREPROCESAMIENTO DE documentos.txt


Primero extraemos las palabras de documentos_final.txt


In [139]:
filename = "documentos_final.txt"
file = open(filename, 'rt',encoding='utf-8-sig')
text = file.read()
file.close()
import re
words = re.split(r"\n", text)
doc_words = ""
i = 0
for w in words:
    split = re.split(r"\|", w)
    if len(split) == 1:
        continue
    elif len(split) == 4:
        #doc_data[i]['d_num'] = split[0]
        doc_words += split[2] + " "
    doc_words += split[1] + " "
    i = i + 1
print(doc_words[:1000])

 preliminary report international algebraic language   extraction root repeated subtraction digital computer   technique department matrix program scheme   glossary computer engineering programming terminology  square root approximation   computer inspection procedure   glossary computer engineering programming terminology  equivalence transformation program scheme   proposal uncol   glossary computer engineering programming terminology  problem programming communication changing machine proposed solution part   error estimation runge kutta procedure   glossary computer engineering programming terminology  problem programming communication changing machine proposed solution part   recursive curve fitting technique   secant modification newton method   programming arithmetic operation   simple automatic coding system   glossary computer engineering programming terminology  technique discussed applied iterative procedure solution equation accelerates rate convergence iteration converges 

Hacemos lo mismo con querys_final.txt

In [140]:
filename = "querys_final.txt"
file = open(filename, 'rt',encoding='utf-8-sig')
text = file.read()
file.close()
import re
words = re.split(r"\n", text)
query_words = ""
i = 0
for w in words:
    split = re.split(r"\|", w)
    if len(split) == 1:
        continue
    elif len(split) == 4:
        #doc_data[i]['d_num'] = split[0]
        query_words += split[2] + " "
    query_words += split[1] + " "
    i = i + 1
print(query_words[:1000])

 article exist deal tss time sharing system operating system ibm computer  interested article written prieve udo pooch prieve pooch  intermediate language construction multi targeted compiler tcoll  interested mechanism communicating disjoint process possibly exclusively distributed environment description complete mechanism implementation opposed theoretical work abstract problem remote procedure call message passing example interest  paper design implementation editing interface window manager command interpreter essential issue human interface design view improvement user efficiency effectiveness satisfaction  interested article robotics motion planning geometric combinatorial aspect interested dynamic arm motion  interested distributed algorithm concurrent program process communicate synchronize message passing area interest include fault tolerance technique understanding correctness algorithm  addressing scheme resource network resource addressing network operating system  securit

Unimos y separamos todas las palabras (omitiendo vacías)

In [141]:
all_words = [x for x in (doc_words + " " + query_words).split(" ") if x]
print(all_words[:100])
print(str(len(all_words)))

['preliminary', 'report', 'international', 'algebraic', 'language', 'extraction', 'root', 'repeated', 'subtraction', 'digital', 'computer', 'technique', 'department', 'matrix', 'program', 'scheme', 'glossary', 'computer', 'engineering', 'programming', 'terminology', 'square', 'root', 'approximation', 'computer', 'inspection', 'procedure', 'glossary', 'computer', 'engineering', 'programming', 'terminology', 'equivalence', 'transformation', 'program', 'scheme', 'proposal', 'uncol', 'glossary', 'computer', 'engineering', 'programming', 'terminology', 'problem', 'programming', 'communication', 'changing', 'machine', 'proposed', 'solution', 'part', 'error', 'estimation', 'runge', 'kutta', 'procedure', 'glossary', 'computer', 'engineering', 'programming', 'terminology', 'problem', 'programming', 'communication', 'changing', 'machine', 'proposed', 'solution', 'part', 'recursive', 'curve', 'fitting', 'technique', 'secant', 'modification', 'newton', 'method', 'programming', 'arithmetic', 'opera

Ahora buscaremos reducir el vocabulario existente

In [142]:
from collections import Counter

c = Counter(all_words)
#print(c.items())
labels, values = zip(*c.items())
print(labels[:10])
print(values[:10])
print(len(labels))

('preliminary', 'report', 'international', 'algebraic', 'language', 'extraction', 'root', 'repeated', 'subtraction', 'digital')
(19, 95, 7, 69, 831, 10, 67, 7, 9, 135)
7627


In [143]:
# vemos los datos en deciles
import statistics
deciles = statistics.quantiles(values, n=10)
print(deciles)

[1.0, 1.0, 1.0, 2.0, 2.0, 3.0, 6.0, 10.0, 24.0]


Viendo la distribución de los datos en deciles, vemos que la casi mitad de las palabras del vocabulario aparecen en frecuencias cercanas a 1 y 2.

Se optará por eliminar solamente las palabras con 1 ocurrencia en el vocabulario.

In [144]:
reduced_words = {k: v for k, v in c.items() if v == 1}
print(len(reduced_words))
print("We deleted " + str(len(labels) - len(reduced_words)) + " words from the vocabulary")

2877
We deleted 4750 words from the vocabulary


Vemos que se eliminaron 4,750 palabras, que a mi consideración fueron demasiadas, seguiremos revisando el vocabulario existente.

Analizaremos la distribución del largo de los términos del vocabulario.

In [145]:
len_freq_dict = dict()
for w in labels:
    len_freq_dict[len(w)] = 0
for w in labels:
    len_freq_dict[len(w)] = len_freq_dict[len(w)] + 1
len_freq_list = (sorted(len_freq_dict.items()))
len_freq_list

[(1, 7),
 (2, 174),
 (3, 282),
 (4, 643),
 (5, 635),
 (6, 846),
 (7, 929),
 (8, 973),
 (9, 962),
 (10, 800),
 (11, 566),
 (12, 360),
 (13, 234),
 (14, 107),
 (15, 58),
 (16, 28),
 (17, 12),
 (18, 7),
 (19, 1),
 (20, 3)]

In [146]:
x = len_freq_list[0][1] +len_freq_list[1][1] + len_freq_list[2][1] + len_freq_list[3][1]
x / len(labels) * 100

14.501114461780517

Aquí ya vemos que las palabras con 1, 2, 3 y 4 caracteres que existen en el vocabulario conforman el 14.5% del diccionario, por lo que estas serán las que se eliminarán

In [147]:
reduced_words = {k: v for k, v in c.items() if len(k) > 4}
print("We deleted " + str(len(labels) - len(reduced_words)) + " words from the vocabulary")

We deleted 1106 words from the vocabulary


Por último, las palabras serán escritas a vocabulario.txt y vocabulario_reducido.txt

In [148]:
voc = ', '.join(list(labels))
reduced_voc = ', '.join(list(reduced_words.keys()))
with open('vocabulario.txt', 'w') as f:
    f.write(voc)
    f.close()
with open('vocabulario_reducido.txt', 'w') as f:
    f.write(reduced_voc)
    f.close()